In [1]:
# import importants
import pandas as pd
import pickle
import re
import numpy as np

In [2]:
with open('pickled_df', 'rb') as f:
    df_1140 = pickle.load(f)

In [24]:
# df_1140.info(verbose = True, null_counts = True)

# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
df_1140

In [4]:
# Strip columns of whitespce
df_1140.columns = df_1140.columns.str.strip()

# second HDD column
df_1140.columns.values[-10] = 'HDDx'

# whacky rows
# \n12 Cores\n8 Performance-cores 4 Efficient-cores\n
# \n10 Cores\n6 Performance-cores 4 Efficient-cores\n
# df_1140['Brand'].loc[df_1140['Brand'] == '\n10 Cores\n6 Performance-cores 4 Efficient-cores\n']
df_1140 = df_1140.drop([45, 52, 167, 168])

In [5]:
pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
# df_1140.info(verbose=True, null_counts=True)

In [6]:
# search for numbers in the strings
number = '\d+[.,]?\d*'

units_MB = ['MB']
units_GB = ['GB', 'TB']
units_GHz = ['GHz']
units_W = ['W']

cases_MB = fr'({number})(?:[\s\d\-\+\/]*)(?:{units_MB})'
cases_GB = fr'({number})(?:[\s\d\-\+\/]*)(?:{units_GB})'
cases_GHz = fr'({number})(?:[\s\d\-\+\/]*)(?:{units_GHz})'
cases_W = fr'({number})(?:[\s\d\-\+\/]*)(?:{units_W})'

pattern_MB = re.compile(cases_MB)
pattern_GB = re.compile(cases_GB)
pattern_GHz = re.compile(cases_GHz)
pattern_W = re.compile(cases_W)

In [7]:
# clean Price
df_1140['Price'] = df_1140['Price'].apply(lambda x: float(x))
df_1140['Price'] = df_1140['Price'].fillna(df_1140['Price'].mean())
df_1140['Price'] = df_1140['Price'].apply(lambda x: round(x, 2))

In [8]:
# clean Memory
df_1140['Memory'] = df_1140['Memory'].fillna('0 GB')
df_1140['Memory'] = df_1140['Memory'].apply(lambda x: str(x))

df_1140['RAM_GB'] = df_1140['Memory'].apply(lambda x: pattern_GB.findall(x))

df_1140['RAM_GB'] = df_1140['RAM_GB'].apply(lambda x: ['0'] if x == [] else x)
df_1140['RAM_GB'] = df_1140['RAM_GB'].apply(lambda x: max(x) if len(x) > 1 else x[0])
df_1140['RAM_GB'] = df_1140['RAM_GB'].replace('0', np.nan)
df_1140['RAM_GB'] = df_1140['RAM_GB'].fillna('8')
df_1140['RAM_GB'] = df_1140['RAM_GB'].apply(lambda x: int(x))

In [9]:
# clean SSD
df_1140['SSD'] = df_1140['SSD'].fillna('0 GB')
df_1140['SSD'] = df_1140['SSD'].replace('No', '0 GB')
df_1140['SSD'] = df_1140['SSD'].apply(lambda x: str(x))

df_1140['SSD_GB'] = df_1140['SSD'].apply(lambda x: pattern_GB.findall(x))

df_1140['SSD_GB'] = df_1140['SSD_GB'].apply(lambda x: ['0'] if x == [] else x)
df_1140['SSD_GB'] = df_1140['SSD_GB'].apply(lambda x: max(x) if len(x) > 1 else x[0])
df_1140['SSD_GB'] = df_1140['SSD_GB'].apply(lambda x: float(x))
df_1140['SSD_GB'] = df_1140['SSD_GB'].apply(lambda x: x*1000 if x < 10 else x)
df_1140['SSD_GB'] = df_1140['SSD_GB'].apply(lambda x: int(x))

In [10]:
# clean HDD
df_1140['HDD'] = df_1140['HDD'].fillna('0 GB')
df_1140['HDD'] = df_1140['HDD'].replace('No', '0 GB')
df_1140['HDD'] = df_1140['HDD'].apply(lambda x: str(x))

df_1140['HDD_GB'] = df_1140['HDD'].apply(lambda x: pattern_GB.findall(x))

df_1140['HDD_GB'] = df_1140['HDD_GB'].apply(lambda x: ['0'] if x == [] else x)
df_1140['HDD_GB'] = df_1140['HDD_GB'].apply(lambda x: max(x) if len(x) > 1 else x[0])
df_1140['HDD_GB'] = df_1140['HDD_GB'].apply(lambda x: int(x))
df_1140['HDD_GB'] = df_1140['HDD_GB'].apply(lambda x: x*1000 if x < 11 else x)

In [11]:
# clean Power Supply
df_1140['Power Supply'] = df_1140['Power Supply'].fillna('0 W')
df_1140['Power Supply'] = df_1140['Power Supply'].apply(lambda x: str(x))

df_1140['Power_W'] = df_1140['Power Supply'].apply(lambda x: pattern_W.findall(x))

df_1140['Power_W'] = df_1140['Power_W'].apply(lambda x: ['0'] if x == [] else x)
df_1140['Power_W'] = df_1140['Power_W'].apply(lambda x: max(x) if len(x) > 1 else x[0])
df_1140['Power_W'] = df_1140['Power_W'].apply(lambda x: int(x))

In [12]:
# clean CPU Speed
df_1140['CPU Speed'] = df_1140['CPU Speed'].fillna('0.0 GHz')
df_1140['CPU Speed'] = df_1140['CPU Speed'].apply(lambda x: str(x))

df_1140['Processor_GHz'] = df_1140['CPU Speed'].apply(lambda x: pattern_GHz.findall(x))

df_1140['Processor_GHz'] = df_1140['Processor_GHz'].apply(lambda x: ['0.0'] if x == [] else x)
df_1140['Processor_GHz'] = df_1140['Processor_GHz'].apply(lambda x: x[1] if len(x) > 1 else x[0])
df_1140['Processor_GHz'] = df_1140['Processor_GHz'].replace('0.0', np.nan)
df_1140['Processor_GHz'] = df_1140['Processor_GHz'].apply(lambda x: float(x))
df_1140['Processor_GHz'] = df_1140['Processor_GHz'].fillna(df_1140['Processor_GHz'].mean())
df_1140['Processor_GHz'] = df_1140['Processor_GHz'].apply(lambda x: round(x, 2))

In [13]:
# clean L3 Cache per CPU
df_1140['L3 Cache Per CPU'] = df_1140['L3 Cache Per CPU'].fillna('0 MB')
df_1140['L3 Cache Per CPU'] = df_1140['L3 Cache Per CPU'].apply(lambda x: str(x))

df_1140['L3_Cache_MB'] = df_1140['L3 Cache Per CPU'].apply(lambda x: pattern_MB.findall(x))

df_1140['L3_Cache_MB'] = df_1140['L3_Cache_MB'].apply(lambda x: ['0'] if x == [] else x)
df_1140['L3_Cache_MB'] = df_1140['L3_Cache_MB'].apply(lambda x: x[0])
df_1140['L3_Cache_MB'] = df_1140['L3_Cache_MB'].apply(lambda x: int(x))
df_1140['L3_Cache_MB'] = df_1140['L3_Cache_MB'].replace(0, df_1140['L3_Cache_MB'].mean())
df_1140['L3_Cache_MB'] = df_1140['L3_Cache_MB'].apply(lambda x: int(x))

In [14]:
# clean Memory Capacity
df_1140['Memory Capacity'] = df_1140['Memory Capacity'].fillna('0 GB')
df_1140['Memory Capacity'] = df_1140['Memory Capacity'].apply(lambda x: '0 GB' if 'GB' not in x else x)
df_1140['Memory Capacity'] = df_1140['Memory Capacity'].apply(lambda x: str(x))

df_1140['Memory_Capacity_GB'] = df_1140['Memory Capacity'].apply(lambda x: pattern_GB.findall(x))

df_1140['Memory_Capacity_GB'] = df_1140['Memory_Capacity_GB'].apply(lambda x: ['0'] if x == [] else x)
df_1140['Memory_Capacity_GB'] = df_1140['Memory_Capacity_GB'].apply(lambda x: x[1] if len(x) > 1 else x[0])
df_1140['Memory_Capacity_GB'] = df_1140['Memory_Capacity_GB'].apply(lambda x: int(x))
df_1140['Memory_Capacity_GB'] = df_1140['Memory_Capacity_GB'].replace(0, df_1140['Memory_Capacity_GB'].median())

In [15]:
# clean Release Date
df_1140['Date First Available'] = df_1140['Date First Available'].fillna('0')

df_1140['Release_Date'] = df_1140['Date First Available'].str.split(' ')

df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: x[-1])
df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: '0' if '\"' in x else x)
df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: '0' if 'ports' in x else x)
df_1140['Release_Date'] = df_1140['Release_Date'].replace('', '0')
df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: int(x))
df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: df_1140['Release_Date'].median() if x < 2000 else x)
df_1140['Release_Date'] = df_1140['Release_Date'].apply(lambda x: int(x))

In [16]:
# clean Brand
df_1140['Brand'] = df_1140['Brand'].fillna('Not_Listed')
df_1140['Brand'] = df_1140['Brand'].str.strip()
df_1140['Brand'] = df_1140['Brand'].str.lower()
df_1140['Brand'] = df_1140['Brand'].str.replace(' ', '')

In [17]:
df_1140['Brand'].value_counts()

# common_zips=df_comb.dealer_zip.value_counts().index[df_comb.dealer_zip.value_counts()>100]
# df_comb['zips_clean']=df_comb.dealer_zip.apply(lambda x:x if x in common_zips else 'Other')

df_brands = pd.get_dummies(df_1140['Brand'],drop_first=True, columns = 'Price')
df_brands

,acer,adamantcustom,adenine,apple,asus,clx,cobratype,cyberpowerpc,dell,helix,hoengager,hp,ibuypower,lenovo,msi,not_listed,optiplex,pricerightcomputers,skytech,tsingtongfang,velocitymicro,velztorm,viprtech,xoticpc,yeyian
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1136,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1137,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1138,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
numeric_cols = ['Price', 'Power_W', 'RAM_GB', 'SSD_GB', 'HDD_GB', 'Processor_GHz', 'L3_Cache_MB', 'Memory_Capacity_GB', 'Release_Date']
df_n = df_1140[numeric_cols]

df_n = df_n.join(df_brands)

# made new df for brand, merge back into df_n

In [19]:
with open('pickled_df_cleaned', 'wb') as f:
    pickle.dump(df_n,f)

In [20]:
# End of Data Cleaning
# Start Step_3 - Linear Regression